In [76]:
import polars as pl
import datetime as dt

In [77]:
dt_from = dt.date(2020, 1, 1)
dt_to = dt.date(2030, 12, 31)

In [78]:
date_series = pl.date_range(
    dt_from,
    dt_to,
    '1d',
    eager=True
)

In [84]:
df = date_series.to_frame('date')

In [80]:
mapping_weekday_short = {
    1: 'Mo',
    2: 'Di',
    3: 'Mi',
    4: 'Do',
    5: 'Fr',
    6: 'Sa',
    7: 'So'
}
mapping_weekday_long = {
    1: 'Montag',
    2: 'Dienstag',
    3: 'Mittwoch',
    4: 'Donnerstag',
    5: 'Freitag',
    6: 'Samstag',
    7: 'Sonntag'
}
mapping_month_short = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mrz',
    4: 'Apr',
    5: 'Mai',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Okt',
    11: 'Nov',
    12: 'Dez'
}
mapping_month_long = {
    1: 'Januar',
    2: 'Februar',
    3: 'März',
    4: 'April',
    5: 'Mai',
    6: 'Juni',
    7: 'Juli',
    8: 'August',
    9: 'September',
    10: 'Oktober',
    11: 'November',
    12: 'Dezember'
}

In [ ]:
df_date = (df
            .with_columns(
               Datum_DIN = pl.col('date').dt.to_string('iso'),
               Tag_im_Monat = pl.col('date').dt.day(),
               Tag_im_Jahr = pl.col('date').dt.ordinal_day(),
               Wochentag_numerisch = pl.col('date').dt.weekday(),
               Wochentag_kurz = pl.col('date').dt.weekday().replace_strict(mapping_weekday_short),
               Wochentag_lang = pl.col('date').dt.weekday().replace_strict(mapping_weekday_long),
               Quartal_numerisch = pl.col('date').dt.quarter(),
               Quartal_lang = pl.col('date').dt.quarter().cast(pl.String) + '. Quartal',
               Quartal_kurz = 'Q' + pl.col('date').dt.quarter().cast(pl.String),
               Quartal_Jahr = pl.col('date').dt.year().cast(pl.String) + '-Q' + pl.col('date').dt.quarter().cast(pl.String),
               Quartalsende = pl.date(pl.col('date').dt.year(), pl.col('date').dt.quarter() * 3, 1).dt.month_end(), #end of quarter
               Quartalsanfang = pl.date(pl.col('date').dt.year(), (pl.col('date').dt.quarter() * 3) - 2, 1).dt.month_start(), #begin of quarter
               Kalenderwoche_numerisch = pl.col('date').dt.week(),
               Kalenderwoche_kurz = 'KW ' + pl.col('date').dt.week().cast(pl.String).str.zfill(2),
               Kalenderwoche_Jahr = pl.col('date').dt.year().cast(pl.String) + ' | KW ' + pl.col('date').dt.week().cast(pl.String).str.zfill(2),
               Kalenderwoche_Anfang = pl.col('date').dt.offset_by('-' + ((pl.col('date').dt.weekday()) - 1).cast(pl.String) + 'd'),
               Kalenderwoche_Ende = pl.col('date').dt.offset_by((7 - (pl.col('date').dt.weekday())).cast(pl.String) + 'd'),
               Jahr = pl.col('date').dt.year(),
               Jahr_Anfang = pl.date(pl.col('date').dt.year(), 1, 1),
               Jahr_Ende = pl.date(pl.col('date').dt.year(), 12, 31),
               Monat_numerisch = pl.col('date').dt.month(),
               Monat_kurz = pl.col('date').dt.month().replace_strict(mapping_month_short),
               Monat_lang = pl.col('date').dt.month().replace_strict(mapping_month_long),
               Monat_Anfang = pl.date(pl.col('date').dt.year(), pl.col('date').dt.month(), 1),
            )
            .with_columns(
               Quartal_Tage = (pl.col('date') - pl.col('Quartalsanfang')).dt.total_days() + 1, #day count in current quarter
               Jahr_Tage = (pl.col('date') - pl.col('Jahr_Anfang')).dt.total_days() + 1,
               Monat_Jahr_kurz = pl.col('Monat_kurz') + ' ' + pl.col('Jahr').cast(pl.String),
               Monat_Jahr_lang = pl.col('Monat_lang') + ' ' + pl.col('Jahr').cast(pl.String),
               Monat_Jahr_numerisch = pl.col('Monat_numerisch').cast(pl.String).str.zfill(2) + '/' + pl.col('Jahr').cast(pl.String),
               Monat_Ende = pl.col('Monat_Anfang').dt.month_end(),               
            )
            .with_columns(                
                Monat_Anzahl_Tage = pl.col('Monat_Ende').dt.day(),
                pk_Datum = 'date'                
            )
            .select(pl.exclude('date'))
           )

# to add in view
# Heute
# Folgetag
# Vortag
# Ist_Monatserster
# Ist_Monatsletzter
# Ist_Diese_Woche
# Ist_Dieses_Quartal
# Ist_Dieses_Jahr
# Ist_Quartalsanfang
# Ist_Quartalsende


pk_Datum,Wochentag_kurz,Kalenderwoche_Anfang,Kalenderwoche_Ende
date,str,date,date
2020-01-01,"""Mi""",2019-12-30,2020-01-05
2020-01-02,"""Do""",2019-12-30,2020-01-05
2020-01-03,"""Fr""",2019-12-30,2020-01-05
2020-01-04,"""Sa""",2019-12-30,2020-01-05
2020-01-05,"""So""",2019-12-30,2020-01-05
…,…,…,…
2030-12-27,"""Fr""",2030-12-23,2030-12-29
2030-12-28,"""Sa""",2030-12-23,2030-12-29
2030-12-29,"""So""",2030-12-23,2030-12-29
